In [1]:
%%time
!pip install --target=/kaggle/working --extra-index-url=https://pypi.nvidia.com "cudf-cu12==25.2.*" "cuml-cu12==25.2.*"
!rm -rf /kaggle/working/numpy*

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 161.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.8/522.8 kB 191.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 151.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 169.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 164.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 148.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
!pip install scikit-learn==1.5.2 koolbox


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
import cudf
cudf.__version__

'25.02.02'

In [4]:

import cuml
cuml

<module 'cuml' from '/kaggle/working/cuml/__init__.py'>

In [5]:
%load_ext cudf.pandas

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

VER=1

In [6]:
# from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.special import logit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import joblib
import shutil
import optuna
import glob
import json

warnings.filterwarnings('ignore')

In [7]:
train=pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv',index_col='id')
test=pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv',index_col='id')
train2=pd.read_csv('/kaggle/input/bank-marketing-dataset-full/bank-full.csv',delimiter=";")
augment_data = pd.read_csv('/kaggle/input/data-augmentation/df_synth.csv')

train.shape

(750000, 17)

In [8]:
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [9]:
train2.y=train2.y.map({"no":0,'yes':1})

In [10]:
augment_data.columns=train.columns

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df=pd.DataFrame()
dict_lab=dict()
for col in train2.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(train2[col])
    label_dict = {label: idx for idx, label in enumerate(le.classes_)}
    dict_lab[col]=label_dict

for key in dict_lab.keys():
    augment_data[key]=augment_data[key].map({k:v for v,k in dict_lab[key].items()})


In [12]:
train3=augment_data[augment_data.y==1]
train = pd.concat([train,train2,train3],axis=0,ignore_index=True).sample(frac=1.0).reset_index(drop=True)
print("Combined Train shape", train.shape)

Combined Train shape (988936, 17)


In [13]:
train.y.value_counts()/len(train)

y
0    0.707259
1    0.292741
Name: count, dtype: float64

In [14]:
 
duplicates = train[train.duplicated(keep=False)]
duplicates

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y


In [15]:
CATS =[t for t in train.select_dtypes(exclude=np.number).columns ]
print(f"There are {len(CATS)} categorical columns:")
print( CATS )
print(f"There are 1 numerical column:")
print( ["balance"] )

There are 9 categorical columns:
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
There are 1 numerical column:
['balance']


In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
import numpy as np


In [17]:
def data_process(df): # Source: https://www.kaggle.com/code/haohuanchen/ps-s5e8-lightgb-model-a-simple-starter
    df = df.copy()
    
    def many_no(x):
        if x['default']=='no' and x['housing']=='no' and x['loan']=='no':
            return 21
        if x['default']=='no' and x['housing']=='no'\
        or x['default']=='no' and x['loan']=='no'\
        or x['housing']=='no' and x['loan']=='no':
            return 7
        if x['default']=='no' or x['housing']=='no' or x['loan']=='no':
            return 3
        return 0
    
    df['many_no']  = df.apply(lambda x: many_no(x), axis=1).astype("str")
    
    return df
train = data_process(train) 
test = data_process(test) 


In [18]:
def create_advanced_features(df):
    df = df.copy()
    
    # Basic binary features
    df['balance_positive'] = (df['balance'] > 0).astype(int)
    df['has_previous'] = (df['previous'] > 0).astype(int)
    df['duration_long'] = (df['duration'] > 300).astype(int)
    df['duration_very_long'] = (df['duration'] > 600).astype(int)
    df['campaign_multiple'] = (df['campaign'] > 2).astype(int)
    df['campaign_high'] = (df['campaign'] > 5).astype(int)
    
    # Enhanced numerical transformations
    df['log_duration'] = np.log1p(df['duration'])
    df['sqrt_age'] = np.sqrt(df['age'])
    df['log_balance'] = np.sign(df['balance']) * np.log1p(np.abs(df['balance']))
    df['age_squared'] = df['age'] ** 2
    df['duration_squared'] = df['duration'] ** 2
    df['campaign_squared'] = df['campaign'] ** 2
    
    # Detailed age binning
    df['age_bin'] = pd.cut(df['age'], bins=[0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 100], labels=False)
    
    # Duration binning
    df['duration_bin'] = pd.qcut(df['duration'], q=15, labels=False, duplicates='drop')
    
    # Balance binning
    df['balance_bin'] = pd.qcut(df['balance'], q=10, labels=False, duplicates='drop')
    
    # Feature interactions
    df['age_duration'] = df['age'] * df['duration']
    df['age_campaign'] = df['age'] * df['campaign']
    df['balance_duration'] = df['balance'] * df['duration']
    df['campaign_duration'] = df['campaign'] * df['duration']
    df['age_balance'] = df['age'] * df['balance']
    df['previous_campaign'] = df['previous'] * df['campaign']
    
    # Ratios and rates
    df['duration_per_campaign'] = df['duration'] / (df['campaign'] + 1)
    df['balance_per_age'] = df['balance'] / (df['age'] + 1)
    df['duration_per_age'] = df['duration'] / (df['age'] + 1)
    df['campaign_per_age'] = df['campaign'] / (df['age'] + 1)
    
    # Contact-related features
    df['contact_success_rate'] = df['previous'] / (df['campaign'] + 1)
    df['pdays_binned'] = pd.cut(df['pdays'], bins=[-2, -1, 0, 50, 100, 200, 300, 1000], labels=False)
    df['has_pdays'] = (df['pdays'] != -1).astype(int)
    
    # Seasonal features
    month_map = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
    df['month_num'] = df['month'].map(month_map)
    df['quarter'] = ((df['month_num'] - 1) // 3) + 1
    df['is_summer'] = df['month'].isin(['jun', 'jul', 'aug']).astype(int)
    df['is_winter'] = df['month'].isin(['dec', 'jan', 'feb']).astype(int)

    
    return df

cat_train =   create_advanced_features(train)
cat_test =   create_advanced_features(test)
# test_clean =   create_advanced_features(test_clean)


In [19]:
train

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,many_no
0,39,blue-collar,married,secondary,no,876,yes,no,unknown,7,may,110,2,-1,0,unknown,0,7
1,39,management,married,tertiary,no,0,yes,no,cellular,18,nov,52,1,-1,0,unknown,0,7
2,33,management,single,tertiary,no,2031,yes,no,unknown,29,mar,824,7,0,0,unknown,1,7
3,24,blue-collar,single,secondary,no,544,yes,yes,cellular,18,may,523,2,-1,0,unknown,0,3
4,34,services,divorced,secondary,no,4,yes,no,unknown,6,may,201,1,-1,0,unknown,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988931,35,services,single,unknown,no,753,no,no,unknown,4,jun,108,1,-1,0,unknown,0,21
988932,33,management,single,tertiary,no,5359,no,no,cellular,19,aug,253,1,183,3,success,1,21
988933,37,technician,divorced,secondary,no,-18,no,no,cellular,21,aug,870,4,-1,0,unknown,0,21
988934,41,blue-collar,married,primary,no,-1,yes,no,unknown,6,may,117,3,-1,0,unknown,0,7


In [20]:
Num=[]
for i in train.select_dtypes(np.number).columns.tolist():
    print(i,train[i].nunique())
    if i !="y" and i !="balance":
        Num.append(i)

age 78
balance 14319
day 31
duration 1968
campaign 52
pdays 725
previous 52
y 2


In [21]:
COMBO = []
for i,c in enumerate(CATS):
    #print(f"{c}, ",end="")
    combine = pd.concat([train[c],test[c]],axis=0)
    combine,_ = pd.factorize(combine)
    train[c] = combine[:len(train)]
    test[c] = combine[len(train):]
    n = f"{c}_wc"
    train[n] = train[c]*100 + train["balance"]
    test[n] = test[c]*100 + test["balance"]
    COMBO.append(n)
print()
print(f"We engineer {len(COMBO)} new columns!")
print( COMBO )


We engineer 9 new columns!
['job_wc', 'marital_wc', 'education_wc', 'default_wc', 'housing_wc', 'loan_wc', 'contact_wc', 'month_wc', 'poutcome_wc']


In [22]:
FEATURES = CATS + ["balance"] + COMBO + Num
print(f"We now have {len(FEATURES)} columns:")
print( FEATURES )

We now have 25 columns:
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'balance', 'job_wc', 'marital_wc', 'education_wc', 'default_wc', 'housing_wc', 'loan_wc', 'contact_wc', 'month_wc', 'poutcome_wc', 'age', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [23]:
# STATISTICS TO AGGEGATE FOR OUR FEATURE GROUPS
STATS = ["mean","std","count","nunique","median","min","max","skew"]
STATS2 = ["mean","std"]

In [24]:
scores = {}
oof_pred_probs = {}
test_pred_probs = {}

In [25]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

import keras
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers


2025-08-16 23:49:24.535038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755388164.899907      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755388165.000485      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [26]:
def build_model(cat_features, num_features):
    
    x_input_cats = layers.Input(shape=(len(cat_features),))
    embs = []
    for j in range(len(cat_features)):
        e = layers.Embedding(cat_features_card[j], int(np.ceil(np.sqrt(cat_features_card[j]))))
        x = e(x_input_cats[:,j])
        x = layers.Flatten()(x)
        embs.append(x)
        
    x_input_nums = layers.Input(shape=(len(num_features),))
    
    x = layers.Concatenate(axis=-1)(embs+[x_input_nums]) 
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[x_input_cats,x_input_nums], outputs=x)
    return model

In [27]:
cat_features_card = []
for f in CATS:
        cat_features_card.append(train[f].nunique())
    
 

In [28]:
oof_pred_probs = {}
test_pred_probs = {}

In [29]:
%%time
FOLDS = 7

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
def train_model ( mode='xgb'):
    oof = np.zeros((len(train)))
    pred = np.zeros((len(test)))
    save_data=dict()
  
        
    # OUTER K FOLD
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        print(f"### OUTER Fold {i+1} ###")
    
        X_train = train.loc[train_index,FEATURES+['y']].reset_index(drop=True).copy()
        y_train = train.loc[train_index,'y']
    
        X_valid = train.loc[test_index,FEATURES].reset_index(drop=True).copy()
        y_valid = train.loc[test_index,'y']
    
        X_test = test[FEATURES].reset_index(drop=True).copy()
    
        # INNER K FOLD (TO PREVENT LEAKAGE WHEN USING PRICE)
        kf2 = KFold(n_splits=FOLDS, shuffle=True, random_state=42)   
        for j, (train_index2, test_index2) in enumerate(kf2.split(X_train)):
            print(f" ## INNER Fold {j+1} (outer fold {i+1}) ##")
    
            X_train2 = X_train.loc[train_index2,FEATURES+['y']].copy()
            X_valid2 = X_train.loc[test_index2,FEATURES].copy()
    
            ### FEATURE SET 1 (uses price) ###
            col = "balance"
            tmp = X_train2.groupby(col).y.agg(STATS)
            tmp.columns = [f"TE1_wc_{s}" for s in STATS]
            X_valid2 = X_valid2.merge(tmp, on=col, how="left")
            for c in tmp.columns:
                X_train.loc[test_index2,c] = X_valid2[c].values
    
            ### FEATURE SET 2 (uses price) ###
            for col in COMBO:
                tmp = X_train2.groupby(col).y.agg(STATS2)
                tmp.columns = [f"TE2_{col}_{s}" for s in STATS2]
                X_valid2 = X_valid2.merge(tmp, on=col, how="left")
                for c in tmp.columns:
                    X_train.loc[test_index2,c] = X_valid2[c].values
    
        ### FEATURE SET 1 (uses price) ###
        col = "balance"
        tmp = X_train.groupby(col).y.agg(STATS)
        tmp.columns = [f"TE1_wc_{s}" for s in STATS]
        X_valid = X_valid.merge(tmp, on=col, how="left")
        X_test = X_test.merge(tmp, on=col, how="left")
    
        ### FEATURE SET 2 (uses price) ###
        for col in COMBO:
            tmp = X_train.groupby(col).y.agg(STATS2)
            tmp.columns = [f"TE2_{col}_{s}" for s in STATS2]
            X_valid = X_valid.merge(tmp, on=col, how="left")
            X_test = X_test.merge(tmp, on=col, how="left")
    
        ### FEATURE SET 3 (does not use price) ###
        for col in CATS:
            col2 = "balance"
            tmp = X_train.groupby(col)[col2].agg(STATS2)
            tmp.columns = [f"FE3_{col}_wc_{s}" for s in STATS2]
            X_train = X_train.merge(tmp, on=col, how="left")
            X_valid = X_valid.merge(tmp, on=col, how="left")
            X_test = X_test.merge(tmp, on=col, how="left")
    
        # CONVERT TO CATS SO XGBOOST RECOGNIZES THEM
        if mode =='NN':
            for col in X_train.drop('y',axis=1).columns.tolist():
                X_train[col]=X_train[col].fillna(X_train[col].mean())
                X_valid[col]=X_valid[col].fillna(X_valid[col].mean())
                X_test[col]=X_test[col].fillna(X_test[col].mean())
        else:
            X_train[CATS] = X_train[CATS].astype("category")
            X_valid[CATS] = X_valid[CATS].astype("category")
            X_test[CATS] = X_test[CATS].astype("category")
     
 
        # DROP PRICE THAT WAS USED FOR TARGET ENCODING
        X_train = X_train.drop(['y'],axis=1)
    
  
        # Build model
        if mode ==  'xgb':
  
            model = XGBClassifier(
                max_depth=6,
                colsample_bytree=0.5,
                subsample=0.957720586533685,
                n_estimators=10_000,
                learning_rate=0.013548962701658028,
                enable_categorical=True,
                min_child_weight=4,
                # colsample_bytree=0.5094806768077214,
                eval_metric="auc",  # use AUC as evaluation metric
                use_label_encoder=False,
                device="cuda",  # uses GPU
            )
        elif mode ==  'cat':
 
            model = CatBoostClassifier(**{'random_state': 42,
                                 'cat_features': CATS,
                                  'eval_metric': "Logloss",
                                 'n_estimators' : 10000,
                                 'learning_rate': 0.06524873965257823,
                                 'l2_leaf_reg': 0.8867612905712001,
                                 'bagging_temperature': 0.1317347791955057,
                                 'random_strength': 0.9922857768340815,
                                 'depth': 7,
                                 'min_data_in_leaf': 8,
                                 'task_type': "GPU",
                                 "loss_function":'CrossEntropy',
                                 "use_best_model":True,
                                 "allow_writing_files":False,
                                 })

        elif mode ==  'lgbm':
              
              model = LGBMClassifier(**{'random_state': 42,
                              'early_stopping_round': 100,
                              'verbose': -1,
                              'n_estimators': 10000,
                              'metric': 'AUC',
                              'objective': 'binary',
                              'max_depth': 16,
                              'learning_rate': 0.007366917567300051,
                              'min_child_samples': 164,
                              'subsample': 0.9022880020285295,
                              'colsample_bytree': 0.4213201532077694,
                              'num_leaves': 122, 
                              'reg_alpha': 1.083996192298843,
                              'reg_lambda': 0.0700057221912873
                              })
           
        elif mode ==  'rfc':
 
            model = HistGradientBoostingClassifier(
                learning_rate=0.02,
                max_iter=10_000,
                max_depth=6,
                max_leaf_nodes=31,
                min_samples_leaf=10,
                l2_regularization=1.0,
                early_stopping=True,
                scoring='roc_auc',  # used only when early_stopping=True
                validation_fraction=0.1,
                n_iter_no_change=100,
                categorical_features=CATS,  # only needed if data is NumPy array
                random_state=42,
                verbose=0
            )
        elif mode == 'NN':
                num_features = [f for f in X_train.columns.tolist() if f not in CATS]
            
                X_train_cats = X_train[CATS]
                X_train_nums = X_train[num_features]

                X_val_cats = X_valid[CATS]
                X_val_nums = X_valid[num_features]

                X_test_cats = X_test[CATS]
                X_test_nums = X_test[num_features]

                model = build_model(CATS, num_features)                        
                keras.utils.set_random_seed(42)
                optimizer = keras.optimizers.AdamW(learning_rate=1e-2, weight_decay=1e-3)
                model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['auc'])
                 
               
        # TRAIN MODEL
        COLS = X_train.columns
        if mode =='lgbm':
             
            model.fit(
                X_train[COLS], y_train,
                eval_set=[(X_valid[COLS], y_valid)],
            )
        elif mode =='rfc':
 
       
            model.fit(
                X_train[COLS], y_train,
            )
        elif mode =='NN':
            
             
             model.fit([X_train_cats,X_train_nums], y_train, 
                          validation_data=([X_val_cats, X_val_nums], y_valid),
                          epochs=20,
                          batch_size=512,
                          callbacks=[keras.callbacks.ReduceLROnPlateau(patience=1),
                                     keras.callbacks.EarlyStopping(patience=3)
                                    ])
        elif mode=="cat":
            model.fit(X_train, y_train, eval_set=[(X_valid[COLS], y_valid)], early_stopping_rounds=200)
        else:
            model.fit(
                X_train[COLS], y_train,
                eval_set=[(X_valid[COLS], y_valid)],  
                verbose=300,
            )
        if mode =="NN":
            oof[test_index] = model.predict([X_val_cats, X_val_nums]).squeeze()                      
            pred += model.predict([X_test_cats, X_test_nums]).squeeze() 
            
        else:
            oof[test_index] = model.predict_proba(X_valid[COLS])[:,1]
            pred += model.predict_proba(X_test[COLS])[:,1]
        
    pred /= FOLDS
    oof_pred_probs[mode]=oof
    test_pred_probs[mode]=pred
    return oof_pred_probs ,test_pred_probs 

CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 31.2 µs


In [30]:
# COMPUTE OVERALL CV SCORE
true = train.y.values
for m in ["lgbm","xgb", "cat","rfc","NN" ]:
    
    oof_pred_probs ,test_pred_probs =train_model(mode=m)
    print(f"{m} , done")
    s=roc_auc_score(true,oof_pred_probs[m])
    print(f"=> {m} Overall CV Score = {s}")
    
# => xgb Overall CV Score = 0.9852468139005441
# => cat Overall CV Score = 0.9846821674909663
# => rfc Overall CV Score = 0.9846549244741901

### OUTER Fold 1 ###
 ## INNER Fold 1 (outer fold 1) ##
 ## INNER Fold 2 (outer fold 1) ##
 ## INNER Fold 3 (outer fold 1) ##
 ## INNER Fold 4 (outer fold 1) ##
 ## INNER Fold 5 (outer fold 1) ##
 ## INNER Fold 6 (outer fold 1) ##
 ## INNER Fold 7 (outer fold 1) ##
### OUTER Fold 2 ###
 ## INNER Fold 1 (outer fold 2) ##
 ## INNER Fold 2 (outer fold 2) ##
 ## INNER Fold 3 (outer fold 2) ##
 ## INNER Fold 4 (outer fold 2) ##
 ## INNER Fold 5 (outer fold 2) ##
 ## INNER Fold 6 (outer fold 2) ##
 ## INNER Fold 7 (outer fold 2) ##
### OUTER Fold 3 ###
 ## INNER Fold 1 (outer fold 3) ##
 ## INNER Fold 2 (outer fold 3) ##
 ## INNER Fold 3 (outer fold 3) ##
 ## INNER Fold 4 (outer fold 3) ##
 ## INNER Fold 5 (outer fold 3) ##
 ## INNER Fold 6 (outer fold 3) ##
 ## INNER Fold 7 (outer fold 3) ##
### OUTER Fold 4 ###
 ## INNER Fold 1 (outer fold 4) ##
 ## INNER Fold 2 (outer fold 4) ##
 ## INNER Fold 3 (outer fold 4) ##
 ## INNER Fold 4 (outer fold 4) ##
 ## INNER Fold 5 (outer fold 4) ##
 ## IN

0:	learn: 0.5736211	test: 0.5739835	best: 0.5739835 (0)	total: 393ms	remaining: 1h 5m 34s
1:	learn: 0.5001659	test: 0.5005966	best: 0.5005966 (1)	total: 485ms	remaining: 40m 24s
2:	learn: 0.4311764	test: 0.4317109	best: 0.4317109 (2)	total: 571ms	remaining: 31m 44s
3:	learn: 0.3787797	test: 0.3793475	best: 0.3793475 (3)	total: 656ms	remaining: 27m 18s
4:	learn: 0.3398979	test: 0.3404246	best: 0.3404246 (4)	total: 729ms	remaining: 24m 16s
5:	learn: 0.3143793	test: 0.3148065	best: 0.3148065 (5)	total: 800ms	remaining: 22m 12s
6:	learn: 0.2943419	test: 0.2946617	best: 0.2946617 (6)	total: 871ms	remaining: 20m 43s
7:	learn: 0.2763570	test: 0.2767031	best: 0.2767031 (7)	total: 941ms	remaining: 19m 35s
8:	learn: 0.2639050	test: 0.2641946	best: 0.2641946 (8)	total: 1.01s	remaining: 18m 43s
9:	learn: 0.2530448	test: 0.2533342	best: 0.2533342 (9)	total: 1.08s	remaining: 18m 1s
10:	learn: 0.2445713	test: 0.2448151	best: 0.2448151 (10)	total: 1.16s	remaining: 17m 30s
11:	learn: 0.2361575	test: 0.

0:	learn: 0.5730865	test: 0.5729909	best: 0.5729909 (0)	total: 93.1ms	remaining: 15m 31s
1:	learn: 0.4995481	test: 0.4993109	best: 0.4993109 (1)	total: 183ms	remaining: 15m 13s
2:	learn: 0.4285261	test: 0.4281706	best: 0.4281706 (2)	total: 268ms	remaining: 14m 52s
3:	learn: 0.3835706	test: 0.3830936	best: 0.3830936 (3)	total: 349ms	remaining: 14m 31s
4:	learn: 0.3482460	test: 0.3477753	best: 0.3477753 (4)	total: 418ms	remaining: 13m 55s
5:	learn: 0.3217803	test: 0.3212601	best: 0.3212601 (5)	total: 491ms	remaining: 13m 37s
6:	learn: 0.3010615	test: 0.3005001	best: 0.3005001 (6)	total: 562ms	remaining: 13m 22s
7:	learn: 0.2841182	test: 0.2835324	best: 0.2835324 (7)	total: 631ms	remaining: 13m 7s
8:	learn: 0.2711978	test: 0.2706115	best: 0.2706115 (8)	total: 703ms	remaining: 13m
9:	learn: 0.2595732	test: 0.2589539	best: 0.2589539 (9)	total: 772ms	remaining: 12m 51s
10:	learn: 0.2505778	test: 0.2499258	best: 0.2499258 (10)	total: 840ms	remaining: 12m 43s
11:	learn: 0.2432986	test: 0.24259

0:	learn: 0.5734480	test: 0.5739240	best: 0.5739240 (0)	total: 93ms	remaining: 15m 29s
1:	learn: 0.5002959	test: 0.5010410	best: 0.5010410 (1)	total: 186ms	remaining: 15m 30s
2:	learn: 0.4310396	test: 0.4320835	best: 0.4320835 (2)	total: 267ms	remaining: 14m 49s
3:	learn: 0.3794855	test: 0.3807417	best: 0.3807417 (3)	total: 344ms	remaining: 14m 18s
4:	learn: 0.3405464	test: 0.3419697	best: 0.3419697 (4)	total: 417ms	remaining: 13m 53s
5:	learn: 0.3147620	test: 0.3163274	best: 0.3163274 (5)	total: 490ms	remaining: 13m 36s
6:	learn: 0.2929704	test: 0.2946091	best: 0.2946091 (6)	total: 559ms	remaining: 13m 18s
7:	learn: 0.2772757	test: 0.2790872	best: 0.2790872 (7)	total: 630ms	remaining: 13m 6s
8:	learn: 0.2624220	test: 0.2643387	best: 0.2643387 (8)	total: 701ms	remaining: 12m 58s
9:	learn: 0.2526406	test: 0.2545887	best: 0.2545887 (9)	total: 770ms	remaining: 12m 48s
10:	learn: 0.2442110	test: 0.2462016	best: 0.2462016 (10)	total: 840ms	remaining: 12m 42s
11:	learn: 0.2373958	test: 0.239

0:	learn: 0.5736153	test: 0.5732936	best: 0.5732936 (0)	total: 92.7ms	remaining: 15m 26s
1:	learn: 0.5003831	test: 0.4998486	best: 0.4998486 (1)	total: 187ms	remaining: 15m 34s
2:	learn: 0.4311652	test: 0.4304539	best: 0.4304539 (2)	total: 276ms	remaining: 15m 18s
3:	learn: 0.3800367	test: 0.3791653	best: 0.3791653 (3)	total: 350ms	remaining: 14m 35s
4:	learn: 0.3410160	test: 0.3400545	best: 0.3400545 (4)	total: 422ms	remaining: 14m 3s
5:	learn: 0.3151276	test: 0.3141312	best: 0.3141312 (5)	total: 495ms	remaining: 13m 44s
6:	learn: 0.2934246	test: 0.2923489	best: 0.2923489 (6)	total: 565ms	remaining: 13m 25s
7:	learn: 0.2779583	test: 0.2768348	best: 0.2768348 (7)	total: 633ms	remaining: 13m 10s
8:	learn: 0.2630794	test: 0.2619521	best: 0.2619521 (8)	total: 704ms	remaining: 13m 1s
9:	learn: 0.2533995	test: 0.2522667	best: 0.2522667 (9)	total: 772ms	remaining: 12m 51s
10:	learn: 0.2453064	test: 0.2440873	best: 0.2440873 (10)	total: 849ms	remaining: 12m 50s
11:	learn: 0.2378659	test: 0.23

...

Epoch 1/20


I0000 00:00:1755402625.542003     296 service.cc:148] XLA service 0x7c76c4008800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755402625.543573     296 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1755402625.543596     296 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1755402626.304640     296 cuda_dnn.cc:529] Loaded cuDNN version 90300


  37/1656 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - auc: 0.6468 - loss: 0.8949

I0000 00:00:1755402629.759491     296 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1656/1656 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - auc: 0.8832 - loss: 0.3917 - val_auc: 0.9398 - val_loss: 0.3093 - learning_rate: 0.0100
Epoch 2/20
1656/1656 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - auc: 0.9352 - loss: 0.2923 - val_auc: 0.9396 - val_loss: 0.3648 - learning_rate: 0.0100
Epoch 3/20
1656/1656 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - auc: 0.9440 - loss: 0.2725 - val_auc: 0.9423 - val_loss: 0.4209 - learning_rate: 1.0000e-03
Epoch 4/20
1656/1656 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - auc: 0.9473 - loss: 0.2644 - val_auc: 0.9422 - val_loss: 0.3575 - learning_rate: 1.0000e-04
4415/4415 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step
### OUTER Fold 2 ###
 ## INNER Fold 1 (outer fold 2) ##
 ## INNER Fold 2 (outer fold 2) ##
 ## INNER Fold 3 (outer fold 2) ##
 ## INNER Fold 4 (outer fold 2) ##
 ## INNER Fold 5 (outer fold 2) ##
 ## INNER Fold 6 (outer fold 2) ##
 ## INNER Fold 7 (outer fold 2) ##
Epoch 1/20
1656/1656 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - auc: 0.8810 - lo

In [31]:
X = logit(pd.DataFrame(oof_pred_probs).clip(1e-15, 1-1e-15))
X_test = logit(pd.DataFrame(test_pred_probs).clip(1e-15, 1-1e-15))


In [32]:
from sklearn.metrics import roc_auc_score
from koolbox import Trainer

class CFG:

    
    n_folds = 7
    seed = 42
    
    cv = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
    metric = roc_auc_score
    
    n_optuna_trials = 200

In [33]:
def objective(trial):
    solver_penalty_options = [
        ('liblinear', 'l1'),
        ('liblinear', 'l2'),
        ('lbfgs', 'l2'),
        ('lbfgs', None),
        ('newton-cg', 'l2'),
        ('newton-cg', None),
        ('newton-cholesky', 'l2'),
        ('newton-cholesky', None)
    ]
    solver, penalty = trial.suggest_categorical('solver_penalty', solver_penalty_options)
    
    params = {
        'random_state': CFG.seed,
        'max_iter': 1000,
        'C': trial.suggest_float('C', 0, 1),
        'tol': trial.suggest_float('tol', 1e-6, 1e-2),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'solver': solver,
        'penalty': penalty
    }
    
    
    trainer = Trainer(
        LogisticRegression(**params),
        cv=CFG.cv,
        metric=CFG.metric,
        metric_precision=6,
        use_early_stopping=False,
        verbose=False,
        task="binary",
    )
    trainer.fit(pd.DataFrame(X), true)
    
    return np.mean(trainer.fold_scores)

sampler = optuna.samplers.TPESampler(seed=CFG.seed, multivariate=True, n_startup_trials=CFG.n_optuna_trials // 10)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=CFG.n_optuna_trials, n_jobs=-1)
best_params = study.best_params


[I 2025-08-17 04:01:25,936] A new study created in memory with name: no-name-90439530-56a5-4f5c-8d5d-ed15c23096d4
[I 2025-08-17 04:01:52,393] Trial 2 finished with value: 0.9860243580008345 and parameters: {'solver_penalty': ('liblinear', 'l1'), 'C': 0.3051288101905857, 'tol': 0.0041863220358624945, 'fit_intercept': True, 'class_weight': None}. Best is trial 2 with value: 0.9860243580008345.
[I 2025-08-17 04:01:53,467] Trial 3 finished with value: 0.9861015131114239 and parameters: {'solver_penalty': ('lbfgs', None), 'C': 0.9532072858207207, 'tol': 0.0034678842693025166, 'fit_intercept': False, 'class_weight': 'balanced'}. Best is trial 3 with value: 0.9861015131114239.
[I 2025-08-17 04:01:53,766] Trial 0 finished with value: 0.986110689759573 and parameters: {'solver_penalty': ('liblinear', 'l2'), 'C': 0.3834696234909212, 'tol': 0.005506758338126547, 'fit_intercept': False, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.986110689759573.
[I 2025-08-17 04:01:56,074] Trial 1 

In [34]:
solver, penalty = best_params['solver_penalty']
lr_params = {
    'random_state': CFG.seed,
    'max_iter': 1000,
    'C': best_params['C'],
    'tol': best_params['tol'],
    'fit_intercept': best_params['fit_intercept'],
    'class_weight': best_params['class_weight'],
    'solver': solver,
    'penalty': penalty
}

In [35]:
lr_trainer = Trainer(
    LogisticRegression(**lr_params),
    cv=CFG.cv,
    metric=CFG.metric,
    metric_precision=6,
    use_early_stopping=False,
    task="binary",
)

lr_trainer.fit(pd.DataFrame(X), true)

scores["LogisticRegression"] = lr_trainer.fold_scores
lr_test_pred_probs = lr_trainer.predict(X_test)
print(np.mean(scores["LogisticRegression"]))

Training LogisticRegression

--- Fold 0 - roc_auc_score: 0.986427 - Time: 0.91 s
--- Fold 1 - roc_auc_score: 0.985643 - Time: 0.90 s
--- Fold 2 - roc_auc_score: 0.986160 - Time: 0.88 s
--- Fold 3 - roc_auc_score: 0.986218 - Time: 0.89 s
--- Fold 4 - roc_auc_score: 0.985889 - Time: 0.94 s
--- Fold 5 - roc_auc_score: 0.986313 - Time: 0.87 s
--- Fold 6 - roc_auc_score: 0.986277 - Time: 0.85 s

------ Overall roc_auc_score: 0.986131 - Mean roc_auc_score: 0.986133 ± 0.000253 - Time: 9.34 s
0.9861326486412151


In [36]:
X =pd.DataFrame(X)
X.columns=pd.DataFrame(oof_pred_probs).columns

X_test =pd.DataFrame(X_test)
X_test.columns=pd.DataFrame(test_pred_probs).columns

In [37]:
oof_ens = 0.25 * pd.DataFrame(X)['lgbm'] + 0.4 * pd.DataFrame(X)["xgb"]+ 0.3 * pd.DataFrame(X)['cat'] + 0.25 * pd.DataFrame(X)['rfc']+ 0.05 * pd.DataFrame(X)['NN']
prd_ens = 0.25 * pd.DataFrame(X_test)['lgbm'] + 0.4 * pd.DataFrame(X_test)["xgb"]+ 0.3 * pd.DataFrame(X_test)['cat'] + 0.25 * pd.DataFrame(X_test)['rfc']+ 0.05 * pd.DataFrame(X_test)['NN']
m = roc_auc_score(true, oof_ens)
print(f" CV Score AUC = {m:.3f}")

 CV Score AUC = 0.986


In [38]:
sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
sub['y']=lr_test_pred_probs

sub.to_csv("submission.csv",index=False)
sub.head()

,id,y
0,750000,0.002795
1,750001,0.112405
2,750002,0.000931
3,750003,0.000089
4,750004,0.059556


In [39]:
sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
sub['y']=prd_ens

sub.to_csv("submission_ens.csv",index=False)
sub.head()

,id,y
0,750000,-5.829632
1,750001,-2.224971
2,750002,-6.763720
3,750003,-9.090727
4,750004,-2.793673


In [40]:
lr_ens=0.8*lr_test_pred_probs+0.2*prd_ens
sub = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")
sub['y']=lr_test_pred_probs

sub.to_csv("submission_lr_ens.csv",index=False)
sub.head()

,id,y
0,750000,0.002795
1,750001,0.112405
2,750002,0.000931
3,750003,0.000089
4,750004,0.059556
